In [1]:
from talon.signature import EXTRACTOR_FILENAME, EXTRACTOR_DATA
from talon.signature.learning.classifier import train, init
import talon.signature.learning.helpers as helpers
import talon.quotations as tq
from IPython.core.display import display, HTML
from talon.quotations import register_xpath_extensions

import talon.utils as utils

register_xpath_extensions()

import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_parquet('./data/part-00000.snappy.parquet')
data = data[data['status'] == '2']
data = data.reset_index()
data.head()

,index,id,parent_id,activity_id,created_by_id,created_date,last_modified_date,last_modified_by_id,system_modstamp,text_body,html_body,headers,subject,from_name,from_address,validated_from_address,to_address,cc_address,bcc_address,incoming,has_attachment,status,message_date,is_deleted,reply_to_email_message_id,is_externally_visible
0,1,02s2o00002y8UF7AAM,5002o00002KiCiLAAV,None,005D00000041R4YIAU,2020-01-08T04:23:58.000+0000,2020-01-08 04:45:57,0052o000008NJvqAAG,2020-01-08 06:46:59,"Hello,\n\nWe are from Vietnam factory and supp...","<html><head><meta http-equiv=""content-type"" co...",None,Question,bella@thanhtincandlesco.com,bella@thanhtincandlesco.com,bella@thanhtincandlesco.com,twillsupport@maersk.com,None,None,True,None,2,2020-01-08 04:23:44,None,None,None
1,2,02s2o00002y8lWhAAI,5002o00002KiNooAAF,None,005D00000041R4YIAU,2020-01-08T07:01:49.000+0000,2020-01-08 08:17:43,0052o000009VvDHAA0,2020-01-08 15:05:56,您好，我的账号登录不了，麻烦帮我处理一下，谢谢！“\n\n\n\nThanks and Be...,"<html><head><meta http-equiv=""content-type"" co...",None,账号登录不了,lisa@kstar.com,lisa@kstar.com,lisa@kstar.com,twillsupport@maersk.com,None,None,True,None,2,2020-01-08 07:02:15,None,None,None
2,3,02s2o00002yGCEYAA4,5002o00002KkJ4AAAV,None,005D00000041R4YIAU,2020-01-09T12:05:26.000+0000,2020-01-09 12:11:02,0052o000009bcD0AAI,2020-01-09 12:12:21,DearPlease check and reply\n\n\n\n\n\nSent fro...,"<html xmlns=""http://www.w3.org/1999/xhtml"" xml...",None,Re: drazleather [ ref:_00DD0n6xF._5002o2KkJ...,mohammad draz,drazleather@yahoo.com,drazleather@yahoo.com,twillsupport@maersk.com,yara.almomani@maersk.com,None,True,None,2,2020-01-09 12:05:01,None,None,None
3,6,02s2o00002yZsA4AAK,5002o00002KjkuXAAR,None,005D00000041R4YIAU,2020-01-13T12:27:48.000+0000,2020-01-14 09:46:38,0052o000009bavFAAQ,2020-01-14 09:47:05,"Hi Ms Nha,\n\n \n\nSorry that customer have ca...","<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",None,回覆: 回覆: 回覆: SUBMIT NEW BOOKING FOR CUSTOMER RO...,Judy,judy@omaywood.com,judy@omaywood.com,twillsupport@maersk.com,harley@omaywood.com; wendy.dengvn@outlook.com,None,True,None,2,2020-01-13 12:24:26,None,None,None
4,8,02s2o00002ydLhbAAE,5002o00002L8IRLAA3,None,005D00000041R4YIAU,2020-01-14T15:58:42.000+0000,2020-01-14 15:58:42,005D00000041R4YIAU,2020-01-14 16:17:48,Hi Matt\n\nI’m just trying to book a shipment....,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",None,RE: Problems trying to book a shipment [ re...,Carolynn Young,carolynn@sackmaker.com,carolynn@sackmaker.com,twillsupport@maersk.com,None,None,True,None,2,2020-01-14 15:58:27,None,None,None


In [3]:
data['html_body_quote_removed'] = data.apply(lambda row: tq.extract_from_html(row['html_body']) if row['html_body'] else None, axis=1)
data['text_body_quote_removed'] = data.apply(lambda row:  utils.html_body_to_text_body(row['html_body_quote_removed']) if row['html_body_quote_removed'] else None, axis=1)


data.head()

,index,id,parent_id,activity_id,created_by_id,created_date,last_modified_date,last_modified_by_id,system_modstamp,text_body,html_body,headers,subject,from_name,from_address,validated_from_address,to_address,cc_address,bcc_address,incoming,has_attachment,status,message_date,is_deleted,reply_to_email_message_id,is_externally_visible,html_body_quote_removed,text_body_quote_removed
0,1,02s2o00002y8UF7AAM,5002o00002KiCiLAAV,None,005D00000041R4YIAU,2020-01-08T04:23:58.000+0000,2020-01-08 04:45:57,0052o000008NJvqAAG,2020-01-08 06:46:59,"Hello,\n\nWe are from Vietnam factory and supp...","<html><head><meta http-equiv=""content-type"" co...",None,Question,bella@thanhtincandlesco.com,bella@thanhtincandlesco.com,bella@thanhtincandlesco.com,twillsupport@maersk.com,None,None,True,None,2,2020-01-08 04:23:44,None,None,None,"<html><head><meta http-equiv=""content-type"" co...","b""Hello, \n\nWe are from Vietnam factory and s..."
1,2,02s2o00002y8lWhAAI,5002o00002KiNooAAF,None,005D00000041R4YIAU,2020-01-08T07:01:49.000+0000,2020-01-08 08:17:43,0052o000009VvDHAA0,2020-01-08 15:05:56,您好，我的账号登录不了，麻烦帮我处理一下，谢谢！“\n\n\n\nThanks and Be...,"<html><head><meta http-equiv=""content-type"" co...",None,账号登录不了,lisa@kstar.com,lisa@kstar.com,lisa@kstar.com,twillsupport@maersk.com,None,None,True,None,2,2020-01-08 07:02:15,None,None,None,"<html><head><meta http-equiv=""content-type"" co...",b'\xe9\x8e\xae\xe3\x84\xa5\xe3\x82\xbd\xe9\x94...
2,3,02s2o00002yGCEYAA4,5002o00002KkJ4AAAV,None,005D00000041R4YIAU,2020-01-09T12:05:26.000+0000,2020-01-09 12:11:02,0052o000009bcD0AAI,2020-01-09 12:12:21,DearPlease check and reply\n\n\n\n\n\nSent fro...,"<html xmlns=""http://www.w3.org/1999/xhtml"" xml...",None,Re: drazleather [ ref:_00DD0n6xF._5002o2KkJ...,mohammad draz,drazleather@yahoo.com,drazleather@yahoo.com,twillsupport@maersk.com,yara.almomani@maersk.com,None,True,None,2,2020-01-09 12:05:01,None,None,None,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",b'Dear \nPlease check and reply \nSent from Ya...
3,6,02s2o00002yZsA4AAK,5002o00002KjkuXAAR,None,005D00000041R4YIAU,2020-01-13T12:27:48.000+0000,2020-01-14 09:46:38,0052o000009bavFAAQ,2020-01-14 09:47:05,"Hi Ms Nha,\n\n \n\nSorry that customer have ca...","<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",None,回覆: 回覆: 回覆: SUBMIT NEW BOOKING FOR CUSTOMER RO...,Judy,judy@omaywood.com,judy@omaywood.com,twillsupport@maersk.com,harley@omaywood.com; wendy.dengvn@outlook.com,None,True,None,2,2020-01-13 12:24:26,None,None,None,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",None
4,8,02s2o00002ydLhbAAE,5002o00002L8IRLAA3,None,005D00000041R4YIAU,2020-01-14T15:58:42.000+0000,2020-01-14 15:58:42,005D00000041R4YIAU,2020-01-14 16:17:48,Hi Matt\n\nI’m just trying to book a shipment....,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",None,RE: Problems trying to book a shipment [ re...,Carolynn Young,carolynn@sackmaker.com,carolynn@sackmaker.com,twillsupport@maersk.com,None,None,True,None,2,2020-01-14 15:58:27,None,None,None,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",None


In [6]:
# TODO FIX ME -> html_body to text_body seems not to work properly.......
data['has_signature'] = data.apply(lambda row: helpers.has_signature(row['text_body_quote_removed'], row['validated_from_address']) if row['text_body_quote_removed'] else 'Empty', axis=1)


TypeError: a bytes-like object is required, not 'str'

In [ ]:
data['has_signature'].value_counts(dropna=False)

In [ ]:
no_signature = data[data['has_signature'].isnull()].reset_index()

no_signature

## Evaluating the `has_signature` function

In [17]:
error_index = 22
print('Email: ', no_signature['validated_from_address'][error_index])
print(no_signature['validated_text_body'][error_index])
print()
print()
print()
display(HTML(no_signature['validated_html_body'][error_index]))


Email:  antoniocabreraplastics@gmail.com
Si esta bien.

El jue., 17 de septiembre de 2020 4:41 p. m., twillsupport@maersk.com <
twillsupport@maersk.com> escribió:





In [22]:
SIGNATURE_MAX_LINES = 16

def has_signature2(body, sender):
    '''Checks if the body has signature. Returns True or False.'''
    # split signature long lines e.g. Tel: +34 123 456 789 | Fax: +34 123 456 789
    non_empty = helpers.flatten_list([line.split(' | ') for line in body.splitlines() if line.strip()])
    """
    Removing long lines from emails before looping through candidates as sometimes we have:
    >>> Jean Phelippe
    >>> ABC Logistics
    >>> Germany
    >>> Wir arbeiten ausschließlich auf Grundlage der Allgemeinen Deutschen Spediteurbedingungen, ADSp2017.
    >>> This e-mail and any attachments are confidential and may also be privileged. If you are not the named recipient, please notify         the sender immediately and do not disclose the contents to another person[...]
    """
    not_long = [line for line in non_empty if len(line) <= 60]
    candidates = not_long[-SIGNATURE_MAX_LINES:]
    upvotes = 0
    for line in candidates:
        if helpers.contains_sender_names(sender)(line):
            print('Contain sender: ')
            print(line)
            return True
        elif (helpers.binary_regex_search(helpers.RE_RELAX_PHONE)(line) +
              helpers.binary_regex_search(helpers.RE_EMAIL)(line) +
              helpers.binary_regex_search(helpers.RE_URL)(line) == 1):
              print('Contain RELAX_PHONE, EMAIL OR URL: ')
              print(line)
              upvotes += 1
        elif (helpers.binary_regex_search(helpers.RE_SIGNATURE_WORDS)(line) == 1):
            print('Contain RE_SIGNATURE_WORDS: ')
            print(line)
            upvotes += 1
        else:
            print('Nothing found')
            print(line)
    if upvotes > 1:
        return True

In [23]:
error_index = 21

print('Email: ', no_signature['validated_from_address'][error_index])
print(no_signature['validated_text_body'][error_index])
print('----------ANALYSIS----------')
has_signature2(no_signature['validated_text_body'][error_index], no_signature['validated_from_address'][error_index])

Email:  hxop05@h-star.cn
Best?Regards!?
Reiko Huang ??????T：0592-2633732F：0592-2633708Email：hxop05@h-star.cn
Xiamen?Hongxing?Logistcs?Co.,Ltd
厦门鸿星物流有限公司
ADD:5F,5th?Floor,Liansheng?Building,NO.28?Huli?Avenue,Xiamen,Fujian,China
厦门市湖里区湖里大道联盛大厦东5楼K-L（签单资料/正本/发票于航运中心交接外勤小陈T:13400666117,如需寄件直接寄邮箱地址即可）
----------ANALYSIS----------
Contain RE_SIGNATURE_WORDS: 
Best?Regards!?
Nothing found
Xiamen?Hongxing?Logistcs?Co.,Ltd
Nothing found
厦门鸿星物流有限公司


In [111]:
len('Plot No. 64, Sector - 11, CBD Belapur, Navi Mumbai – 400614, India ')

67

In [30]:
#display(HTML(no_signature['html_body'][2]))
error_index = 2
#helpers.has_signature(no_signature['validated_text_body'][error_index], no_signature['validated_from_address'][error_index])
print(no_signature['validated_from_address'][error_index])
print()
[line for line in no_signature['validated_text_body'][error_index].splitlines() if line.strip()]

phuongtth@phaleminerals.com.vn



['Dear Ms Nhã,',
 'B/L thêm giúp em vào phần mổ tả hàng hóa:',
 '-          NOT FOR MEDICINAL USE ',
 '-          HS CODE: 38249990',
 'Các nội dung còn lại ok rồi ạ.',
 '         ********',
 'Thank and Best regard!                                              ',
 'HA PHUONG TRAN',
 'Imp – Exp Department',
 'Mobile   : +84 936 093 337',
 'Skype    : haphuong.mars3',
 'Website :   <http://www.phaleminerals.com.vn/> www.phaleminerals.com.vn',
 'Address :  2th Floor, Tower A, Rivera Park , 69  Vu Trong Phung Str.,  Thanh Xuan Dist., Hanoi, Vietnam']

In [67]:
import email

email.message_from_string(html_example)

'Hi Ms Nha,\n\n \n\nSorry that customer have cancelled below booking , thank you.\n\n \n\nBest Regards\n\nJudy'